## Analysis of Sales across different platform for Olist 📈

## Loading Datasets and libraries in the Jupyter Notebook 📔

In [ ]:
%load_ext autoreload
%autoreload 2

# Importing Data and Libraries

import numpy as np
import pandas as pd
import matplotlib as plt
from data import Olist

In [ ]:
# Loading the required dataset

df_dict = Olist().get_data()
orders = df_dict['orders']
orders = orders.copy()
orders.head()

In [ ]:
# Cleaning of the orders dataset
# Converting date columns to datetime
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])

# Filtering only completed orders
orders_clean = orders[orders['order_status'] == 'delivered'].copy()

print(f"Total Orders: {len(orders_clean):,}")
print(f"Date Range: {orders_clean['order_purchase_timestamp'].min()} to {orders_clean['order_purchase_timestamp'].max()}")
orders_clean.head()

In [ ]:
# Loading additional datasets for analysis
order_items = df_dict['order_items']
products = df_dict['products']
customers = df_dict['customers']
sellers = df_dict['sellers']
product_category = df_dict['product_category_name_translation']

# Merging datasets for comprehensive analysis
sales_data = orders_clean.merge(order_items, on='order_id', how='left')
sales_data = sales_data.merge(products, on='product_id', how='left')
sales_data = sales_data.merge(customers, on='customer_id', how='left')
sales_data = sales_data.merge(sellers, on='seller_id', how='left', suffixes=('_customer', '_seller'))

# Adding product category names in English
sales_data = sales_data.merge(product_category, on='product_category_name', how='left')

print(f"Total Sales Records: {len(sales_data):,}")
sales_data.head()

### 1️⃣ Evolution Over Time - Monthly Sales Trend 

In [ ]:
# Group by month and calculate key metrics
sales_data['order_month'] = sales_data['order_purchase_timestamp'].dt.to_period('M')

monthly_sales = sales_data.groupby('order_month').agg({
    'order_id': 'count',  # Number of orders
    'price': 'sum'        # Total revenue
}).reset_index()

monthly_sales.columns = ['Month', 'Number_of_Orders', 'Total_Revenue']
monthly_sales['Month'] = monthly_sales['Month'].astype(str)

# Visualize
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Orders over time
ax1.plot(monthly_sales['Month'], monthly_sales['Number_of_Orders'], marker='o', linewidth=2)
ax1.set_title('Number of Orders per Month', fontsize=14, fontweight='bold')
ax1.set_xlabel('Month')
ax1.set_ylabel('Number of Orders')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Revenue over time
ax2.plot(monthly_sales['Month'], monthly_sales['Total_Revenue'], marker='o', linewidth=2, color='green')
ax2.set_title('Total Revenue per Month (R$)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Month')
ax2.set_ylabel('Revenue (R$)')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
print(f"• Best Month (Orders): {monthly_sales.loc[monthly_sales['Number_of_Orders'].idxmax(), 'Month']}")
print(f"• Best Month (Revenue): {monthly_sales.loc[monthly_sales['Total_Revenue'].idxmax(), 'Month']}")

### 2️⃣ Product Categories - Top Sellers